In [ ]:
import ipyparallel as ipp
nprocs = 3
rc = ipp.Cluster(engine_launcher_class="mpi", n=nprocs).start_and_connect_sync()

In [ ]:
%%px
from mpi4py import MPI
comm = MPI.COMM_WORLD

print(comm.rank, comm.size)


In [ ]:
%%px
import sys, os
basedir = ''
if "__file__" in globals(): basedir = os.path.dirname(__file__)
sys.path.append(os.path.join(basedir, os.path.pardir, 'python'))
import utils

In [ ]:
%%px
resscale = 5.0
xs = [0.0, 140.0, 240.0, 400.0]
ys = [0.0, -70.0, -120.0, -200.0]
lc_depth = 40
uc_depth = 15
coast_distance = 0
extra_width = 0
sztype = 'continental'
io_depth_1 = 139
A      = 100.0      # age of subducting slab (Myr)
qs     = 0.065      # surface heat flux (W/m^2)
Vs     = 100.0      # slab speed (mm/yr)

from sz_problems.sz_slab import create_slab
from sz_problems.sz_geometry import create_sz_geometry
slab = create_slab(xs, ys, resscale, lc_depth)
geom = create_sz_geometry(slab, resscale, sztype, io_depth_1, extra_width, 
                            coast_distance, lc_depth, uc_depth)

In [ ]:
%%px
mesh, cell_tags, facet_tags = geom.generatemesh()
mesh.topology.index_map(2).size_local, len(cell_tags.values)

In [ ]:
%%px
plotter_mesh = utils.plot_mesh(mesh, tags=cell_tags, gather=True, show_edges=True, line_width=1, clim=[1,5])
utils.plot_geometry(geom, plotter=plotter_mesh, color='green', width=2)
utils.plot_couplingdepth(slab, plotter=plotter_mesh, render_points_as_spheres=True, point_size=10.0, color='green')
utils.plot_show(plotter_mesh)

In [ ]:
%%px
import utils
plotter_mesh = utils.plot_mesh(mesh, tags=cell_tags, gather=False, show_edges=True, line_width=1, clim=[1,5])
utils.plot_geometry(geom, plotter=plotter_mesh, color='green', width=2)
utils.plot_couplingdepth(slab, plotter=plotter_mesh, render_points_as_spheres=True, point_size=10.0, color='green')
utils.plot_show(plotter_mesh)

In [ ]:
%%px
import numpy as np

wedge_rids = tuple(set([v['rid'] for k,v in geom.wedge_dividers.items()]+[geom.wedge_rid]))
slab_rids  = tuple([geom.slab_rid])
crust_rids = tuple(set([v['rid'] for k,v in geom.crustal_layers.items()]))

tdim = 2
cell_imap = mesh.topology.index_map(tdim)
num_cells = cell_imap.size_local + cell_imap.num_ghosts

wedge_submesh, wedge_cell_tags, wedge_facet_tags, wedge_cell_map = \
                            utils.create_submesh(mesh, np.concatenate([cell_tags.find(rid) for rid in wedge_rids]), \
                                                 cell_tags, facet_tags)

slab_submesh, slab_cell_tags, slab_facet_tags, slab_cell_map  = \
                            utils.create_submesh(mesh, np.concatenate([cell_tags.find(rid) for rid in slab_rids]), \
                                                 cell_tags, facet_tags)


In [ ]:
%%px
mesh.comm.rank, wedge_submesh.comm.rank, slab_submesh.comm.rank

In [ ]:
%%px
if wedge_submesh.topology.index_map(wedge_submesh.topology.dim).size_local > 0:
    plotter_mesh = utils.plot_mesh(wedge_submesh, tags=wedge_cell_tags, gather=False, show_edges=True, line_width=1, clim=[1,5])
    utils.plot_geometry(geom, plotter=plotter_mesh, color='green', width=2)
    utils.plot_couplingdepth(slab, plotter=plotter_mesh, render_points_as_spheres=True, point_size=10.0, color='green')
    utils.plot_show(plotter_mesh)

In [ ]:
%%px
if slab_submesh.topology.index_map(slab_submesh.topology.dim).size_local > 0:
    plotter_mesh = utils.plot_mesh(slab_submesh, tags=slab_cell_tags, gather=False, show_edges=True, line_width=1, clim=[1,5])
    utils.plot_geometry(geom, plotter=plotter_mesh, color='green', width=2)
    utils.plot_couplingdepth(slab, plotter=plotter_mesh, render_points_as_spheres=True, point_size=10.0, color='green')
    utils.plot_show(plotter_mesh)

In [ ]:
%%px
if wedge_submesh.topology.index_map(wedge_submesh.topology.dim).size_local > 0:
    plotter_mesh = utils.plot_mesh(wedge_submesh, tags=wedge_facet_tags, gather=False, show_edges=True, line_width=1, clim=[0,10])
    utils.plot_geometry(geom, plotter=plotter_mesh, color='green', width=2)
    utils.plot_couplingdepth(slab, plotter=plotter_mesh, render_points_as_spheres=True, point_size=10.0, color='green')
    utils.plot_show(plotter_mesh)

In [ ]:
%%px
if slab_submesh.topology.index_map(slab_submesh.topology.dim).size_local > 0:
    plotter_mesh = utils.plot_mesh(slab_submesh, tags=slab_facet_tags, gather=False, show_edges=True, line_width=1)
    utils.plot_geometry(geom, plotter=plotter_mesh, color='green', width=2)
    utils.plot_couplingdepth(slab, plotter=plotter_mesh, render_points_as_spheres=True, point_size=10.0, color='green')
    utils.plot_show(plotter_mesh)

In [ ]:
%%px
mesh.comm.rank, wedge_submesh.comm.rank, slab_submesh.comm.rank

In [ ]:
%%px
import ufl
import dolfinx as df


wedge_ds = ufl.Measure("ds", domain=wedge_submesh, subdomain_data=wedge_facet_tags)
print(df.fem.assemble_scalar(df.fem.form(df.fem.Constant(wedge_submesh, df.default_scalar_type(1.0))*wedge_ds((14,)))))

slab_ds = ufl.Measure("ds", domain=slab_submesh, subdomain_data=slab_facet_tags)
print(df.fem.assemble_scalar(df.fem.form(df.fem.Constant(slab_submesh, df.default_scalar_type(1.0))*slab_ds(14))))